In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
import polars as pl

In [2]:
from scripts import df_ct_combine

In [3]:
df_ct_combined = df_ct_combine.combine()

In [4]:
df_ct_combined

,customer_id,articles
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,"[663713001, 541518023, 663713001, 578020002, 7..."
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,"[505221004, 685687003, 685687004, 685687001, 5..."
2,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,"[688873012, 501323011, 598859003, 688873020, 6..."
3,0008968c0d451dbc5a9968da03196fe20051965edde741...,"[531310002, 529841001, 662980002, 496111003, 5..."
4,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,"[501820043, 501820043, 674681001, 671505001, 6..."
...,...,...
1362276,fece2f68864c311a0b5208e2eb735b3dcde7e41461d327...,"[756322001, 822311012, 891763003, 889133001]"
1362277,fee56cc5315dafb35a4490ccc6f711092cae913550c832...,"[903647001, 903647001]"
1362278,ff5b8a8b26bf93a66290e9bd1b73393ac6a58968a78519...,"[913597001, 903420001, 761406001]"
1362279,b4db5e5259234574edfff958e170fe3a5e13b6f146752c...,"[673186001, 717205001, 669713004, 514134001, 5..."


In [5]:
df_a = pd.read_csv("~/CIS_4496_Project/data/raw/articles.csv")
df_c = pd.read_csv("~/CIS_4496_Project/data/raw/customers.csv")
df_t = pd.read_csv("~/CIS_4496_Project/data/raw/transactions_train.csv")

In [51]:
df_a

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105537,953450001,953450,5pk regular Placement1,302,Socks,Socks & Tights,1010014,Placement print,9,Black,...,Socks Bin,F,Menswear,3,Menswear,26,Men Underwear,1021,Socks and Tights,Socks in a fine-knit cotton blend with a small...
105538,953763001,953763,SPORT Malaga tank,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey,A,Ladieswear,1,Ladieswear,2,H&M+,1005,Jersey Fancy,Loose-fitting sports vest top in ribbed fast-d...
105539,956217002,956217,Cartwheel dress,265,Dress,Garment Full body,1010016,Solid,9,Black,...,Jersey,A,Ladieswear,1,Ladieswear,18,Womens Trend,1005,Jersey Fancy,"Short, A-line dress in jersey with a round nec..."
105540,957375001,957375,CLAIRE HAIR CLAW,72,Hair clip,Accessories,1010016,Solid,9,Black,...,Small Accessories,D,Divided,2,Divided,52,Divided Accessories,1019,Accessories,Large plastic hair claw.


In [6]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df_a['detail_desc'].values.astype('U'))

In [7]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [8]:
results = {}
item_results = {}

for idx, row in df_a.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], df_a['article_id'][i]) for i in similar_indices]
    items_only = [df_a['article_id'][i] for i in similar_indices]
    
    results[row['article_id']] = similar_items[1:]
    item_results[row['article_id']] = items_only[1:]
    
print('done!')

done!


In [9]:
def item(id):
    return df_a.loc[df_a['article_id'] == id]['detail_desc'].tolist()[0].split(' - ')[0]

In [10]:
def recommend(item_id, num):
    #print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")
    print(f'Recommending {num} products similar to {item_id}.')
    print("-------")
    recs = results[item_id][:num]
    for rec in recs:
        print(f"{rec[1]}")

In [11]:
#recs = results[item_id][:num]
output = item_results[663713001][:12]
print(output)

[487722001, 487722008, 487722007, 487722005, 650037001, 526823001, 667641001, 700345001, 838324001, 807172001, 635530003, 647826001]


In [36]:
predictions = []
for row in df_ct_combined['articles']:
    first_string = row[0]
    firstProd = int(first_string)
    predictions.append(item_results[firstProd][:12])

In [38]:
submission = pd.DataFrame(columns=['customer_id', 'prediction'])
submission['customer_id'] = df_ct_combined['customer_id']
submission['prediction'] = predictions
submission

,customer_id,prediction
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,"[487722001, 487722008, 487722007, 487722005, 6..."
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,"[505221001, 619110001, 619110002, 584962001, 6..."
2,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,"[695632042, 695632033, 695632032, 695632035, 6..."
3,0008968c0d451dbc5a9968da03196fe20051965edde741...,"[531310002, 782767001, 746861008, 746861009, 7..."
4,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,"[501820045, 501820027, 501820028, 501820041, 5..."
...,...,...
1362276,fece2f68864c311a0b5208e2eb735b3dcde7e41461d327...,"[756322009, 756322006, 756322004, 756322001, 7..."
1362277,fee56cc5315dafb35a4490ccc6f711092cae913550c832...,"[903647002, 901939001, 679724003, 679724001, 8..."
1362278,ff5b8a8b26bf93a66290e9bd1b73393ac6a58968a78519...,"[698051001, 913597001, 640131007, 640131006, 6..."
1362279,b4db5e5259234574edfff958e170fe3a5e13b6f146752c...,"[815567001, 902723001, 638736002, 533570002, 7..."


In [39]:
def convert(predStr):
    return predStr.replace('[', '').replace(']', '').replace(',', '')

submission['prediction'] = submission['prediction'].apply(str).apply(convert)
submission

,customer_id,prediction
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,487722001 487722008 487722007 487722005 650037...
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221001 619110001 619110002 584962001 656063...
2,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,695632042 695632033 695632032 695632035 695632...
3,0008968c0d451dbc5a9968da03196fe20051965edde741...,531310002 782767001 746861008 746861009 746861...
4,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,501820045 501820027 501820028 501820041 501820...
...,...,...
1362276,fece2f68864c311a0b5208e2eb735b3dcde7e41461d327...,756322009 756322006 756322004 756322001 785086...
1362277,fee56cc5315dafb35a4490ccc6f711092cae913550c832...,903647002 901939001 679724003 679724001 855772...
1362278,ff5b8a8b26bf93a66290e9bd1b73393ac6a58968a78519...,698051001 913597001 640131007 640131006 640131...
1362279,b4db5e5259234574edfff958e170fe3a5e13b6f146752c...,815567001 902723001 638736002 533570002 777728...


In [41]:
customers_without_transactions = pd.DataFrame(columns=['customer_id', 'prediction'])
transaction_customers = np.unique(df_t['customer_id'])
all_customers = df_c['customer_id']
customers_without_transactions['customer_id'] = list(set(transaction_customers).symmetric_difference(set(all_customers)))

In [42]:
customers_without_transactions

,customer_id,prediction
0,f59268468f5d90cb017d0e9ff7a00ad8baec76eaf2352a...,NaN
1,b868037265f6336daca3decfb0a5fd96d73b9c22cf5f32...,NaN
2,250e50ea3906760a3e170b7c332e144b1785e91b76da82...,NaN
3,a13fd560d2c65fae8d9793f3d2c01465393fa875c565f3...,NaN
4,6a1aeead7de63371a8b0e324edf9ea611ee54d6ae082ed...,NaN
...,...,...
9694,f8085e136c1d6e029dae234b9430d204cb1bd0bd3b90ed...,NaN
9695,1070b6cecf27a6a69e8505deaca9856e17fcdea1f75cd6...,NaN
9696,24d4a93e95df7996762f709ba3c9bb70b7b10f93a3a06f...,NaN
9697,ee427f3722b9fecf4cc0ec2dbf35f3fd1c9732ec3f38d4...,NaN


In [43]:
df_t['article_id'].value_counts().nlargest(12)


706016001    50287
706016002    35043
372860001    31718
610776002    30199
759871002    26329
464297007    25025
372860002    24458
610776001    22451
399223001    22236
706016003    21241
720125001    21063
156231001    21013
Name: article_id, dtype: int64

In [44]:
popular = df_t['article_id'].value_counts()[:12].index.tolist()
popular

/tmp/ipykernel_17965/3723116339.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  popular = df_t['article_id'].value_counts()[:12].index.tolist()


[706016001,
 706016002,
 372860001,
 610776002,
 759871002,
 464297007,
 372860002,
 610776001,
 399223001,
 706016003,
 720125001,
 156231001]

In [45]:
most_popular = []
for row in customers_without_transactions['customer_id']:
    most_popular.append(popular) 


In [46]:
customers_without_transactions['prediction'] = most_popular
customers_without_transactions

,customer_id,prediction
0,f59268468f5d90cb017d0e9ff7a00ad8baec76eaf2352a...,"[706016001, 706016002, 372860001, 610776002, 7..."
1,b868037265f6336daca3decfb0a5fd96d73b9c22cf5f32...,"[706016001, 706016002, 372860001, 610776002, 7..."
2,250e50ea3906760a3e170b7c332e144b1785e91b76da82...,"[706016001, 706016002, 372860001, 610776002, 7..."
3,a13fd560d2c65fae8d9793f3d2c01465393fa875c565f3...,"[706016001, 706016002, 372860001, 610776002, 7..."
4,6a1aeead7de63371a8b0e324edf9ea611ee54d6ae082ed...,"[706016001, 706016002, 372860001, 610776002, 7..."
...,...,...
9694,f8085e136c1d6e029dae234b9430d204cb1bd0bd3b90ed...,"[706016001, 706016002, 372860001, 610776002, 7..."
9695,1070b6cecf27a6a69e8505deaca9856e17fcdea1f75cd6...,"[706016001, 706016002, 372860001, 610776002, 7..."
9696,24d4a93e95df7996762f709ba3c9bb70b7b10f93a3a06f...,"[706016001, 706016002, 372860001, 610776002, 7..."
9697,ee427f3722b9fecf4cc0ec2dbf35f3fd1c9732ec3f38d4...,"[706016001, 706016002, 372860001, 610776002, 7..."


In [47]:
customers_without_transactions['prediction'] = customers_without_transactions['prediction'].apply(str).apply(convert)
customers_without_transactions

,customer_id,prediction
0,f59268468f5d90cb017d0e9ff7a00ad8baec76eaf2352a...,706016001 706016002 372860001 610776002 759871...
1,b868037265f6336daca3decfb0a5fd96d73b9c22cf5f32...,706016001 706016002 372860001 610776002 759871...
2,250e50ea3906760a3e170b7c332e144b1785e91b76da82...,706016001 706016002 372860001 610776002 759871...
3,a13fd560d2c65fae8d9793f3d2c01465393fa875c565f3...,706016001 706016002 372860001 610776002 759871...
4,6a1aeead7de63371a8b0e324edf9ea611ee54d6ae082ed...,706016001 706016002 372860001 610776002 759871...
...,...,...
9694,f8085e136c1d6e029dae234b9430d204cb1bd0bd3b90ed...,706016001 706016002 372860001 610776002 759871...
9695,1070b6cecf27a6a69e8505deaca9856e17fcdea1f75cd6...,706016001 706016002 372860001 610776002 759871...
9696,24d4a93e95df7996762f709ba3c9bb70b7b10f93a3a06f...,706016001 706016002 372860001 610776002 759871...
9697,ee427f3722b9fecf4cc0ec2dbf35f3fd1c9732ec3f38d4...,706016001 706016002 372860001 610776002 759871...


In [48]:
df_submission_all = submission.append(customers_without_transactions, ignore_index=True)

/tmp/ipykernel_17965/2791921899.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_submission_all = submission.append(customers_without_transactions, ignore_index=True)


In [49]:
df_submission_all

,customer_id,prediction
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,487722001 487722008 487722007 487722005 650037...
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221001 619110001 619110002 584962001 656063...
2,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,695632042 695632033 695632032 695632035 695632...
3,0008968c0d451dbc5a9968da03196fe20051965edde741...,531310002 782767001 746861008 746861009 746861...
4,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,501820045 501820027 501820028 501820041 501820...
...,...,...
1371975,f8085e136c1d6e029dae234b9430d204cb1bd0bd3b90ed...,706016001 706016002 372860001 610776002 759871...
1371976,1070b6cecf27a6a69e8505deaca9856e17fcdea1f75cd6...,706016001 706016002 372860001 610776002 759871...
1371977,24d4a93e95df7996762f709ba3c9bb70b7b10f93a3a06f...,706016001 706016002 372860001 610776002 759871...
1371978,ee427f3722b9fecf4cc0ec2dbf35f3fd1c9732ec3f38d4...,706016001 706016002 372860001 610776002 759871...


In [50]:
df_submission_all.to_csv('submission.csv', index = False)